### Load Data

In [1]:
from src.vectorization import Vectorizer
from src.classification import MessageCategorizer
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.multioutput import MultiOutputClassifier 
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from sklearn.metrics import f1_score
import datetime

ModuleNotFoundError: No module named 'xgboost'

In [ ]:
vectorizer = vectorization.Vectorizer('messages_info.db')

train_data = vectorizer.fit(workers=10)

### Visualization

In [ ]:
X = np.vstack([X_train, X_test])
X.shape

In [ ]:
X_reduced = PCA(n_components=50).fit_transform(X)
visual = TSNE(n_components=2,
              perplexity=X_reduced.shape[0]/y_train.shape[1],
              learning_rate=500,
              early_exaggeration=50,  ### Early exaggertaion or learning_rate are too high
              n_iter=1500,
              metric='cosine',
              verbose=2).fit_transform(X_reduced)

visual = pd.DataFrame(visual, columns=['x', 'y'])

fig, ax = plt.subplots(figsize=(10, 10))
sns.scatterplot(x='x', y='y', data=visual, ax=ax)

### Class Testing

In [ ]:
classifier = MessageCategorizer(train_data)